In [47]:
import pandas as pd
import numpy as np

# 1: tính net promoter
# score >= 0 and <= 6: detractor
# (7,8): passive
# (9,10): promoter
df = pd.read_csv('nps.csv')
df
# 2 : tính churn rate


,event_date,user_id,score
0,2020-01-01 19:00:51,290067447,8
1,2020-01-02 17:34:43,223014806,10
2,2020-01-02 19:11:55,187864388,10
3,2020-01-02 19:27:00,318758952,8
4,2020-01-02 21:42:19,189933311,0
...,...,...,...
3321,2020-12-30 21:24:28,289219785,8
3322,2020-12-31 02:01:19,187133285,7
3323,2020-12-31 05:30:00,202035502,10
3324,2020-12-31 06:12:32,196225609,10


In [48]:
df.dtypes

event_date    object
user_id        int64
score          int64
dtype: object

In [49]:
#1 : điểm -> label
#2 : tháng


In [50]:
# df.event_date.astype('datetime')
df.event_date = pd.to_datetime(df.event_date)

In [51]:
df.event_date.min()

Timestamp('2020-01-01 19:00:51')

In [52]:
df.event_date.max()

Timestamp('2020-12-31 22:09:29')

In [53]:
df['month'] = df.event_date.dt.month
df

,event_date,user_id,score,month
0,2020-01-01 19:00:51,290067447,8,1
1,2020-01-02 17:34:43,223014806,10,1
2,2020-01-02 19:11:55,187864388,10,1
3,2020-01-02 19:27:00,318758952,8,1
4,2020-01-02 21:42:19,189933311,0,1
...,...,...,...,...
3321,2020-12-30 21:24:28,289219785,8,12
3322,2020-12-31 02:01:19,187133285,7,12
3323,2020-12-31 05:30:00,202035502,10,12
3324,2020-12-31 06:12:32,196225609,10,12


In [54]:
# dùng pd.cut chia type
df['type'] = pd.cut(df.score, bins = [-1, 6, 8, 10], labels = ['Detractor', 'Passive', 'Promoter'], right=True)

In [55]:
df

,event_date,user_id,score,month,type
0,2020-01-01 19:00:51,290067447,8,1,Passive
1,2020-01-02 17:34:43,223014806,10,1,Promoter
2,2020-01-02 19:11:55,187864388,10,1,Promoter
3,2020-01-02 19:27:00,318758952,8,1,Passive
4,2020-01-02 21:42:19,189933311,0,1,Detractor
...,...,...,...,...,...
3321,2020-12-30 21:24:28,289219785,8,12,Passive
3322,2020-12-31 02:01:19,187133285,7,12,Passive
3323,2020-12-31 05:30:00,202035502,10,12,Promoter
3324,2020-12-31 06:12:32,196225609,10,12,Promoter


In [56]:
nps = df.groupby(by=['month', 'type'])['score'].count().unstack()
nps

type,Detractor,Passive,Promoter
month,,,
1,31,82,215
2,22,90,161
3,33,72,125
4,36,87,178
5,33,83,191
6,32,80,215
7,48,95,258
8,40,103,256
9,18,68,135


In [57]:
nps.dtypes

type
Detractor    int64
Passive      int64
Promoter     int64
dtype: object

In [58]:
nps['Sum'] = nps.Detractor + nps.Passive + nps.Promoter

In [59]:
nps['NPS'] = (nps.Promoter - nps.Detractor) / nps.Sum
nps

type,Detractor,Passive,Promoter,Sum,NPS
month,,,,,
1,31,82,215,328,0.560976
2,22,90,161,273,0.509158
3,33,72,125,230,0.400000
4,36,87,178,301,0.471761
5,33,83,191,307,0.514658
6,32,80,215,327,0.559633
7,48,95,258,401,0.523691
8,40,103,256,399,0.541353
9,18,68,135,221,0.529412


### 2: Churn Rate
churn rate = churn/ active customers
by month

In [60]:
df1 = pd.read_csv('muscle_labs.csv')
df1

,id,customer_id,end_date,start_date,subscription_period,price
0,5,1337,2014-12-28 01:40:57,2014-10-25 12:39:18,monthly,50
1,28,1338,2014-11-19 01:48:05,2014-10-17 13:47:59,monthly,50
2,44,1339,2014-11-01 20:56:37,2014-10-30 05:50:54,annual,500
3,85,1340,2014-12-09 02:51:57,2013-12-05 14:51:57,annual,500
4,168,1341,2015-11-16 09:19:29,2014-11-13 20:19:23,annual,500
...,...,...,...,...,...,...
10492,18069,12456,2015-01-20 13:54:39,2014-09-19 00:38:00,monthly,50
10493,18068,12457,2015-01-20 13:38:29,2014-10-19 00:23:00,monthly,50
10494,319,12458,2015-01-20 10:16:19,2014-11-18 21:09:16,monthly,50
10495,343,12460,2015-01-20 18:50:56,2014-11-19 05:50:32,monthly,50


In [61]:
df1.nunique()

id                     10497
customer_id            10394
end_date               10332
start_date             10290
subscription_period        2
price                      2
dtype: int64

In [62]:
df1.customer_id.value_counts()

8343     2
1529     2
1509     2
4090     2
4089     2
        ..
5248     1
5249     1
5250     1
5251     1
12461    1
Name: customer_id, Length: 10394, dtype: int64

In [63]:
df1[df1.customer_id == 8343]

,id,customer_id,end_date,start_date,subscription_period,price
6440,18035,8343,2013-03-04 20:23:00,2012-12-31 08:23:00,monthly,50
7042,26938,8343,2015-05-27 13:20:00,2014-05-27 13:20:00,annual,500


In [64]:
df1.dtypes

id                      int64
customer_id             int64
end_date               object
start_date             object
subscription_period    object
price                   int64
dtype: object

In [ ]:
df1.
df1['end'] = df1.end_date.


In [1]:
#active : start date < ngày cuối tháng n, end date > tháng n (sau tháng n)
import datetime as dt
def get_customers(yearmonth):
    year = int(yearmonth/100)
    month = yearmonth
    

In [ ]:
eda

- 

